In [2]:
import pandas as pd 
import numpy as np 

In [3]:
df = pd.read_csv('working_datasets/merged_data.csv')

In [4]:
print(f"Total records: {len(df)}")
print(f"Duplicates: {df.duplicated().sum()}")
print(f"Missing values:\n{df.isnull().sum()}")

Total records: 762084
Duplicates: 213462
Missing values:
date              0
state             0
district          0
pincode           0
bio_age_5_17      0
bio_age_17_       0
demo_age_5_17     0
demo_age_17_      0
age_0_5           0
age_5_17          0
age_18_greater    0
dtype: int64


In [6]:
df = df.drop_duplicates()
print(f"Records after removing duplicates: {len(df)}")

Records after removing duplicates: 548622


In [ ]:
df['date'] = pd.to_datetime(df['date'], format='%d-%m-%Y')
df = df.sort_values('date')

In [13]:
# Check extreme values
print(df[['bio_age_5_17', 'bio_age_17_', 'demo_age_5_17', 'demo_age_17_']].describe())

# Cap outliers at 99th percentile
for col in ['bio_age_5_17', 'bio_age_17_', 'demo_age_5_17', 'demo_age_17_']:
    q99 = df[col].quantile(0.99)
    df[col] = df[col].apply(lambda x: q99 if x > q99 else x)

        bio_age_5_17    bio_age_17_  demo_age_5_17   demo_age_17_
count  548622.000000  548622.000000  548622.000000  548622.000000
mean       11.228819      10.453600       2.222102      20.283982
std        15.901447      13.463385       3.027146      26.930198
min         0.000000       0.000000       0.000000       0.000000
25%         2.000000       2.000000       0.000000       5.000000
50%         6.000000       6.000000       1.000000      11.000000
75%        13.000000      13.000000       3.000000      24.000000
max        95.000000      80.000000      16.000000     161.000000


In [9]:
# Total biometric records
df['total_bio'] = df['bio_age_5_17'] + df['bio_age_17_']

# Total demographic records
df['total_demo'] = df['demo_age_5_17'] + df['demo_age_17_']

# Total enrollment
df['total_enrollment'] = df['age_0_5'] + df['age_5_17'] + df['age_18_greater']

# Biometric success rate (bio records / demo records)
df['bio_success_rate'] = (df['total_bio'] / df['total_demo'] * 100).round(2)
df['bio_success_rate'] = df['bio_success_rate'].replace([np.inf, -np.inf], np.nan)
df['bio_success_rate'] = df['bio_success_rate'].fillna(0)

# Add month and quarter columns
df['month'] = df['date'].dt.month
df['quarter'] = df['date'].dt.quarter

In [14]:
df.to_csv('working_datasets/cleaned_data.csv', index=False)
print("Cleaned data saved!")

Cleaned data saved!
